# Introducción a la ciencia de datos con Python
###  Rafa Caballero

# Caso práctico: carga de ficheros


Puedes usar el siguiente índice para saltar a la sección adecuado


**Índice**

[Descarga de ficheros](#Descarga-de-ficheros)

[Carga de ficheros CSV con Pandas](#Carga-de-ficheros-CSV-con-Pandas)

[Carga de ficheros Excel con Pandas](#Carga-de-ficheros-Excel-con-Pandas)

[Carga de ficheros JSON con Pandas](#Carga-de-ficheros-JSON-con-Pandas)

[Extracción de información de ficheros PDF](#Carga-de-ficheros-PDF)

[Extracción de tablas WEB](#Extracción-de-tablas-WEB)



<a name="Descarga-de-ficheros"></a>
### Descarga de ficheros

Como veremos, en algunos casos las librerías correspondientes nos permitirán cargar directamente datos a partir de un enlace en internet. Sin embargo, en otras ocasiones tendremos que seguir dos pasos

  1.- Descargar el fichero que queramos usando la librería `requests`

  2.- Manipular el fichero, ya en local, mediante la librería adecuada

Vamos a ver un ejemplo muy sencillo de descarga del último fichero con los datos de inflación, donde el punto 2 es en este caso una grabación del fichero como fichero local.

In [ ]:
import requests
url = "https://www.bolsamadrid.es/docs/SBolsas/InformesSB/resumen.pdf"
nombre = "resumen.pdf"
r = requests.get(url, allow_redirects=True) # el fichero queda en la variable r
with open(nombre, 'wb') as f:
    f.write(r.content) # ahora lo grabamos localmente

print(r.status_code)

Se puede comprabar si el fichero se ha descargado bien comprobando el `r.status_code`, que será 200 si todo ha sido correcto.

In [ ]:
print(r.status_code)


*   200: Todo ha ido bien
*   301: La página redirige a otra; esto podemose evitarlo con `allow_redirects=True`
*  400: El servidor no existe o el formato de algún parámetro es erróneo
* 401: Error de autentificación
* 403: No tenemos permisos para acceder al recurso
* 404: El recurso no existe aunque el servidor sí
* 503: El servidor existe pero no puede atender la demanda

Por tanto el proceso ha tenido este aspecto:

<img src="https://www.dataquest.io/wp-content/uploads/2019/09/api-request.svg"></img>


Además, [requests](https://docs.python-requests.org/) permite pasar parámetros, autenticarse con user, password, descargar de forma "perezosa" para ficheros que no caben en memoria, etc.

También puede utilizarse para descargar páginas html y examinarlas extrayendo información. Sin embargo, para esto es preferible utilizar librerías como `BeautifulSoup` o, si se tiene que interaccionar con la página, `Selenium`

La variable devuelta por requests (llamada `r` en este ejemplo) tiene dos partes:

* El mensaje de error o de éxito `r.status_code`

* Los datos en sí, `r.content`

La instrucción

```
    with open(nombre, 'wb') as f:
      f.write(r.content) # ahora lo grabamos localmente
```

graba el contenido (que es el fichero PDF, ahora mismo localizado en memoria) en un fichero local con el nombre que incluya la variable `nombre`


**Ejercicio 1** Descargar el Balance de Actividad del año 2021 disponible en "https://www.bolsasymercados.es/docs/infmercado/2021/esp/IM2021.pdf". Grabar el fichero localmente con nombre "informe21.pdf"

Mostrar el status de la descarga a ver si todo está bien

**Ejercicio 2** Probar a modificar la URL para que corresponda al ejercicio 2022:

https://www.bolsasymercados.es/docs/infmercado/2021/esp/IM2022.pdf

y comprobar que el status nos devuelve un error. Grabar el fichero localmente con nombre "informe22.pdf"


<a name="Carga-de-ficheros-CSV-con-Pandas"></a>
### Carga de ficheros CSV con Pandas

[read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) y [read_excel](https://pythonbasics.org/read-excel/) permiten leer ficheros separados por un carácter y ficheros excel
En ambos casos devuelven un dataframe: una tabla en Pandas que representa el conjunto de datos en memoria


##### read_csv

El principal parámetro es la dirección donde se encuentra el fichero. Algunos parámetros comunes de read_csv

- sep: el separador, por defecto “,”
- Header: para indicar si la primera línea contiene la cabecera (por defecto True)
- Thousands, decimal: separadores de miles y de decimales
- encoding: codificación de caracteres. Deber ser una codificaciones estándar


In [ ]:
import pandas as pd # normalmente pandas se renombra como pd
fichero = 'https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/contaminacionLargo.csv'
df = pd.read_csv(fichero)
df

In [ ]:

df2 = df[(df.fecha>'2019-01-11') & (df.fecha<'2019-02-10')]
df2.index=df2.fecha
df2["CO"].plot()

El resultado es un dataframe; una tabla de pandas que utilizaremos como nuestro principal almacen de datos. Si el fichero no tiene el formato adecuado, por ejemplo, por la codificación, podemos obtener un error

In [ ]:
"""
fichero = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/parocomunidades.csv"
df2 = pd.read_csv(fichero)
"""

In [ ]:
import pandas as pd
fichero = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/parocomunidades.csv"
df2 = pd.read_csv(fichero,encoding="latin1") # o cp1252
df2

Ver https://python.readthedocs.io/en/stable/library/codecs.html#standard-encodings para una lista completa de codificaciones

##### to_csv
La función de pandas [to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv) permite grabar un dataframe en formato csv, debemos recordar 2 cosas principalmente:
- se encuentra dentro del objeto dataframe, por eso pondremos df.to_csv, y no pd.to_csv
- Conviene añadir el parámetro index=False para que no incluya el índice o número de línea

In [ ]:
fichero = r".\parocomunidades.csv"
df2.to_csv(fichero, index=False, encoding="utf-8")

**Ejercicio 3**: cargar el fichero que acabamos de grabar y mostrarlo en el notebook

In [ ]:
# solución


<a name="Carga-de-ficheros-Excel-con-Pandas"></a>
### Carga de ficheros Excel con Pandas

##### read_excel
La lectura con pd.read_excel utiliza internamente otro librería `xlrd` que habra que instalar sino está en el sistema. Este ejemplo carga un fichero con valores bursátiles

In [ ]:
import pandas as pd
fichero='https://github.com/RafaelCaballero/tdm/raw/master/datos/raw_open.xlsx'
df = pd.read_excel(fichero)
df

En el caso de múltiples páginas obtendremos un vector de dataframes. En este ejemplo cargamos datos de accidentes por distritos en la ciudad de Madrid. Se trata de un "libro" excel con hojas de nombre '2009', ..., '2016'

Ojo: para manejar esta posibilidad necesitaremos una versión actualizada de xlrd

In [ ]:
#!pip install --upgrade xlrd

In [ ]:
import pandas as pd
fichero = "https://github.com/RafaelCaballero/tdm/raw/master/datos/madrid/accidentes_madrid_2009_2016.xlsx"
df_acc = pd.read_excel(fichero,sheet_name='2016')
df_acc

La grabación de ficheros excel se realiza con la función de pandas to_excel, asociada al dataframe. Esta función depende a su vez de dos librerías


    - xlwt para grabar ficheros .xls  (formato hasta Excel2003)
    - openpyxl para ficheros .xlsx (Excel2007 o posterior).



In [ ]:
df_acc.to_excel("./accidentes.xlsx")

**Ejercicio 5**
Cargar el fichero situado en https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/ciudades_ejemplo.csv
como un dataframe Pandas, y grabarlo a continuación en formato excel.

In [ ]:
# solución


<a name="Carga-de-ficheros-JSON-con-Pandas"></a>
### Lectura de ficheros JSON con Pandas

In [ ]:
url = "https://ucmdrive.ucm.es/s/ZfHGwYfi2zzJgq2/download/labs_co2.json"
import pandas as pd
df = pd.read_json(url)
df

**Ejercicio** Mostrar en qué fecha y hora el laboratorio 5 tuvo mayor temperatura

<a name="Carga-de-ficheros-PDF"></a>
### Extracción de información de ficheros PDF

Este es un tema complejo porque va a depender mucho del formato del interno del PDF. En los casos más sencillos podremos extraer por ejemplo tablas de datos con el paquete tabula-py

In [ ]:
!pip install tabula-py
#!pip install tabulate

In [ ]:
from tabula import read_pdf

fichero = "./resumen.pdf"

# extraer tablas del fichero
df = read_pdf(fichero,pages=1)[0]  # 0 porque nos quedamos con la primera tabla

df

In [ ]:
# Configurar pandas para no usar notación exponencial y mostrar números completos
pd.set_option('display.float_format', '{:.2f}'.format)

# lo grabamos como csv
fichero = "tabla.csv"
df.to_csv(fichero,index=False)

# leer el fichero tabla.csv teniendo indicando que no tiene cabecera y el formato de los números es el de España
df2 = pd.read_csv(fichero, header=None,thousands='.', decimal=',')
df2.columns = ["Índice", "Anterior", "Cierre", "Dif", "Dif %", "Medio", "Capitalización €"]
df2

Ahora podemos grabar el fichero en formato CSV

In [ ]:
df2.to_csv("resumen.csv")

Proximamente veremos como manipular esta información, de momento calculemos por ejemplo la media de la columna Dif

In [ ]:
df2.Dif.mean()

**Ejercicio 6** Grabar el fichero en formato excel, con nombre "resumen.xlsx"

In [ ]:
# solución


<a name="Extracción-de-tablas-WEB"></a>
### Extracción de tablas WEB

El método `read_html`lee todas las tablas en una página web

In [ ]:
import requests
import pandas as pd

url = 'https://datosmacro.expansion.com/paro-epa/espana-comunidades-autonomas'
df_list = pd.read_html(url)
df = df_list[-1] # la última tabla
df